In [ ]:
from config import MyTansforms,MyDataset
from data_loader import MyDataLoading_color,MyDataLoading_four_channel
import pytorch_lightning as pl
import pandas as pd
from myModelsAll import MyEfficientNet_3channel
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
logger_dir = "/panfs/jay/groups/27/kersten/gejji003/Shadows/trained_models/round1/logger/"
checkpt_dirpath = "/panfs/jay/groups/27/kersten/gejji003/Shadows/trained_models/round1/checkpoint/"
base_path = "/panfs/jay/groups/27/kersten/gejji003/Shadows/data/images_rendered_try2/" # loaction of train test
default_string = base_path # location of contact & no_contact

file_path_train = base_path +"trainData.csv"
file_path_val = base_path +"valData.csv"
file_path_test = base_path +"testData.csv"


In [ ]:
train_df = pd.read_csv(file_path_train)
val_df = pd.read_csv(file_path_val)
test_df = pd.read_csv(file_path_test)

In [ ]:
theDataLoader = MyDataLoading_color
color_channel = "colors"
shadow_channel = "cast_shadow"

In [ ]:
dataset = MyDataset(train_df,val_df,test_df,color_channel,shadow_channel,theDataLoader)

In [ ]:
model = MyEfficientNet_3channel(num_classes = 2, input_channels=3)

In [ ]:
logger = TensorBoardLogger(save_dir=logger_dir, name="my_experiment")
early_stopping_callback = EarlyStopping(
    monitor='val_loss',  # The metric to monitor for early stopping (e.g., validation loss)
    patience=20,           # Number of epochs with no improvement after which training will be stopped
    mode='min',            # 'min' means we want to minimize the monitored metric, 'max' means maximize
)
model_checkpoint_callback = ModelCheckpoint(
                                    dirpath=checkpt_dirpath,  # Directory to save checkpoints
                                    filename='model-{epoch:02d}-{val_loss:.2f}',  # File name format
                                    save_top_k=3,  # Save the top 3 best models
                                    monitor='val_loss',  # Monitor validation loss
                                    mode='min',  # Choose 'min' or 'max' depending on the monitored metric
                                    )
# Create a Lightning Trainer
trainer = pl.Trainer(max_epochs=100, accelerator = "gpu",logger=logger, log_every_n_steps=1,
                     callbacks=[model_checkpoint_callback, early_stopping_callback]
        )



In [ ]:
trainer.fit(model, datamodule = dataset)